In [7]:
from utils.spark_config import get_spark_session
from configs.paths import ROOT_DIR

MODEL_DIR = ROOT_DIR / "models"
spark = get_spark_session()
spark

In [10]:
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml import PipelineModel

pipeline_model = PipelineModel.load(f"{MODEL_DIR}/churn_pipeline")

pipeline_model

PipelineModel_4d1bc5146566

In [11]:
churn = spark.read.format("delta").load(f"{ROOT_DIR}/data/gold/customer_churn_gold")
churn.show(5)

+----------+-----+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+
|CustomerID|Churn|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneSer

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
outcomes = pipeline_model.transform(churn).select("CustomerId", "label", "prediction")


# AUC (area under ROC)
binary_eval = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
auc = binary_eval.evaluate(outcomes)
print(f"AUC: {auc}")

# Accuracy
multiclass_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = multiclass_eval.evaluate(outcomes)
print(f"Accuracy: {accuracy}")

25/08/21 12:17:52 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


AUC: 0.7642039651813171


25/08/21 12:17:53 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


Accuracy: 0.8417235494880546
